라이브러리 설치

In [ ]:
# !pip install langchain-teddynote arrow vllm langchain-community rank_bm25 qdrant_client langchain-qdrant guardrails-ai torch ragas faiss-cpu

# RAGAS 평가 준비
---

In [2]:
# LangSmith 추적 설정 (응답 결과 저장을 위함)
import os
from langchain_teddynote import logging

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGSMITH_API_KEY"] = "YOUR_API_KEY"
os.environ["LANGSMITH_PROJECT"] = "LANGCHAIN_PROJECT"

logging.langsmith("LANGCHAIN_PROJECT")

LangSmith 추적을 시작합니다.
[프로젝트명]
LANGCHAIN_PROJECT


In [3]:
# 테스트 데이터셋 확인
import pandas as pd

df = pd.read_csv("notice.me")
print(df.ground_truth[104])

안녕하세요! 🌿
1. 8월 1일 진행되는 파이널 프로젝트와 수료식 참석 여부 조사가 진행 중이며, 마지막 교육 일정의 도시락 제공을 위해 7월 25일(금) 오후 1시까지 구글폼으로 참여 신청을 해주세요.
2. 파이널 프로젝트 예선전 발표 자료는 계획서·결과 보고서·시연영상을 포함하여 7월 30일(수) 오후 5시까지 구글폼으로 제출해야 하며, 본선 자료는 여기에 프로젝트 완료 보고서를 추가하여 구글 드라이브 폴더에 제출해야 합니다.
3. 제출 폴더와 필수 양식 드라이브 링크가 제공되며, 작성 중 궁금한 사항은 운영팀이나 사무실에 문의할 수 있습니다.
4. 6단위기간 종료에 따른 출석률 메일이 발송되며, 이의 신청은 7월 25일(금) 오후 3시까지 DM으로 할 수 있습니다. 훈련 장려금 신청부터 지급까지는 약 한 달이 걸리며, 계좌 정보는 고용24에서 확인하고 문의는 8월 25일 이후 가능합니다.
5. 예선전 발표 순서 추첨은 7월 30일(수) 오전 10시에 타운홀에서 진행되고 발표 시간은 팀당 17분입니다. 예선전은 7월 31일(목) 오전 9시부터 진행되며, 상위 5팀이 본선에 진출하고 우수 발표팀에게는 특별상이 주어집니다.
🙏 모두 파이널 프로젝트와 수료식 준비 잘하시어 좋은 결과 얻으시길 바랍니다.


In [4]:
from datasets import Dataset
import pandas as pd

# 기존 DataFrame(df)을 HuggingFace Dataset으로 단순 변환
test_dataset = Dataset.from_pandas(df)
print(test_dataset)

# RAGAS 평가를 위해 새로운 DataFrame 생성
df_for_ragas = pd.DataFrame({
    "question": df["question"],                # 기존 질문
    "answer": df["ground_truth"],              # LLM 응답 대신 Ground Truth 사용 가능
    "contexts": df["context"].apply(lambda x: [x]),  # context를 리스트로 변환
    "ground_truth": df["ground_truth"]         # 평가 기준 정답 유지
})

# HuggingFace Dataset으로 다시 변환
# 이제 RAGAS에서 바로 사용 가능한 형태로 준비 완료
test_dataset = Dataset.from_pandas(df_for_ragas)

# 변환된 Dataset의 첫 번째 샘플 확인
print(test_dataset[0])


Dataset({
    features: ['question', 'context', 'ground_truth', 'evolution_type', 'meta', 'domain', 'difficulty', 'language', 'episode_done'],
    num_rows: 110
})
{'question': '자격증 응시료는 어떻게 환급받나요?', 'answer': '안녕하세요! 🌿\n1. 교육 과정과 관련된 자격증 취득 시 응시료 전액을 수료 후 1개월 내 환급받을 수 있으며, 지원 대상은 해당 과정 수료자입니다.\n2. 합격증과 매출전표를 제출해야 하며, 민간 자격증은 지원되지 않습니다.\n3. 관련 내용은 문서에서 확인해주시기 바랍니다.\n🙏 추가로 궁금한 사항이 있으면 언제든 문의해주세요', 'contexts': ['교육 과정과 관련된 자격증 취득 시 응시료 전액을 수료 후 1개월 내 환급받을 수 있으며, 지원 대상은 해당 과정 수료자입니다. 합격증과 매출전표를 제출해야 하며, 민간 자격증은 지원되지 않습니다.'], 'ground_truth': '안녕하세요! 🌿\n1. 교육 과정과 관련된 자격증 취득 시 응시료 전액을 수료 후 1개월 내 환급받을 수 있으며, 지원 대상은 해당 과정 수료자입니다.\n2. 합격증과 매출전표를 제출해야 하며, 민간 자격증은 지원되지 않습니다.\n3. 관련 내용은 문서에서 확인해주시기 바랍니다.\n🙏 추가로 궁금한 사항이 있으면 언제든 문의해주세요'}


# 서비스 전체 실행 후 RAGAS 평가 시작
---

In [ ]:
import asyncio
import pandas as pd
from datasets import Dataset
import uuid

# 결과 저장 리스트
answers = []

# 질문 순회하며 응답 생성
async def generate_answers():
    for i, row in df.iterrows():
        ans = await ragas_chat_service(
            question=row["question"],
            request_id=str(uuid.uuid4()),
            userId=123  # 사용자 ID는 테스트용 고정값
        )
        answers.append(ans)

# 비동기 실행
await generate_answers()

# DataFrame에 응답 컬럼 추가
df["answer"] = answers

# contexts 컬럼 생성 (RAGAS 요구사항: 리스트)
df["contexts"] = df["context"].apply(lambda x: [x])

# Hugging Face Dataset 변환
dataset = Dataset.from_pandas(df[["question", "answer", "contexts", "ground_truth"]])

print(dataset[0])

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"

sample_dataset = test_dataset.select(range(20))

from ragas import evaluate
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
)

result = evaluate(
    dataset=sample_dataset,
    metrics=[
        context_precision,
        faithfulness,
        answer_relevancy,
        context_recall,

    ],
)
result

In [19]:
print(result)

{'context_precision': 0.9500, 'faithfulness': 0.6121, 'answer_relevancy': 0.1592, 'context_recall': 0.4475}
